In [1]:
%cd /N/dc2/projects/lifebid/Paolo/code/re_dwi/sandbox

/N/dc2/projects/lifebid/Paolo/code/re_dwi/sandbox


In [2]:
sub = '00'

#### Setting Filenames

In [56]:
dicom_dwi = 'input/sub-%s/dwi' % sub
dicom_t1w = 'input/sub-%s/anat' % sub

dwi_dir = 'sub-%s/dwi' % sub
dwi_src = 'sub-%s_dwi' % sub
!mkdir -p {dwi_dir}

dwi_pre = 'derivatives/preprocess/sub-%s' % sub
!mkdir -p {dwi_pre}

b0_src = 'sub-%s_var-b0src_dwi' % sub
b0_otsu = 'sub-%s_var-b0otsu_dwi' % sub
dwi_nodif = 'sub-%s_var-otsu_dwi_mask' % sub
dwi_ecc = 'sub-%s_var-ecc_dwi' % sub
b0_ecc = 'sub-%s_var-b0ecc_dwi' % sub

t1w_dir = 'sub-%s/anat' % sub
t1w_src = 'sub-%s_T1w.nii.gz' % sub
!mkdir -p {t1w_dir}

t1w_pre = 'derivatives/preprocess/sub-%s' % sub
!mkdir -p {t1w_pre}
t1w_bet = 'sub-%s_var-bet_T1w.nii.gz' % sub
t1w_mas = 'sub-%s_var-bet_T1w_mask.nii.gz' % sub
t1w_5tt = 'sub-%s_var-fsl_dtissue' % sub

t1w_fsl_wmm = 'sub-%s_var-fsl_space-T1w_class-wm_dtissue.nii.gz' % sub
b0_fsl_omat = 'sub-%s_var-fsl_affine.mat' % sub
dwi_fsl_wmm = 'sub-%s_var-fsl_space-dwi_class-wm_dtissue.nii.gz' % sub

fs_dir = 'derivatives/freesurfer/sub-%s' % sub
!mkdir -p {fs_dir}
fs_aa_src = '%s/freesurfer/mri/aparc+aseg' % fs_dir
fs_mask_src = '%s/freesurfer/mri/brainmask' % fs_dir
fs_mask = 'sub-%s_var-fs_T1w_mask.nii.gz' % sub
fs_t1w = 'sub-%s_var-fs_T1w.nii.gz' % sub
t1w_fs_aa = 'sub-%s_var-fs_dtissue.nii.gz' % sub
t1w_fs_wmm = 'sub-%s_var-fs_space-T1w_class-wm_dtissue.nii.gz' % sub
b0_fs_omat = 'sub-%s_var-fs_affine.mat' % sub
dwi_fs_wmm = 'sub-%s_var-fs_space-dwi_class-wm_dtissue.nii.gz' % sub

trk_dir = 'derivatives/tractography/sub-%s' % sub
!mkdir -p {trk_dir}
dti = 'sub-%s_var-mrx_DTI' % sub
fa = 'sub-%s_var-mrx_FA' % sub
sf = 'sub-%s_var-mrx_SF' % sub
rf = 'sub-%s_var-mrx_response.txt' % sub
odf = 'sub-%s_var-mrx_ODF' % sub
tck_prob = 'sub-%s_var-prob' % sub

roi_dir = 'derivatives/fmri_roi/sub-%s' % sub
roi_all = 'sub-%s_lab-all_roi' % sub
roi_all_dwi = 'sub-%s_space-dwi_lab-all_roi' % sub
roi_all_dwi_exp = 'sub-%s_space-dwi_var-exp_lab-all_roi' % sub
roi_1 = 'sub-%s_lab-Pdx_roi.nii.gz' % sub
roi_2 = 'sub-%s_lab-PFdx_roi.nii.gz' % sub
roi_3 = 'sub-%s_lab-Psx_roi.nii.gz' % sub
roi_4 = 'sub-%s_lab-PFsx_roi.nii.gz' % sub
roi_sgm = 'sub-%s_sgm.mif' % sub

con_dir = 'derivatives/connectome/sub-%s' % sub
!mkdir -p {con_dir}
con_csv = 'sub-%s_var-mrx_connectome.csv' % sub

#### Import and convert diffusion data

In [ ]:
tmpdir = '_tmp_dcm2nii'
!mkdir -p {tmpdir}

!dcm2nii -f y -e n -p n -a n -d n -g n -i n -o {tmpdir} {dicom_dwi}

fname = !basename {tmpdir}/*.bval
fname = fname[0].strip('.bval')
!mv {tmpdir}/{fname}.nii {dwi_dir}/{dwi_src}.nii
!gzip -f {dwi_dir}/{dwi_src}.nii
!mv {tmpdir}/{fname}.bval {dwi_dir}/{dwi_src}.bvals
!mv {tmpdir}/{fname}.bvec {dwi_dir}/{dwi_src}.bvecs
!rm -r {tmpdir}

In [ ]:
!mrconvert {dwi_dir}/{dwi_src}.nii.gz {dwi_dir}/{dwi_src}.mif \
    -fslgrad {dwi_dir}/{dwi_src}.bvecs {dwi_dir}/{dwi_src}.bvals \
    -datatype float32 -stride 0,0,0,1 -nthreads 5 -force -quiet

Diffusion brain mask

In [ ]:
!dwiextract  {dwi_dir}/{dwi_src}.mif - -shell 0 -quiet | mrmath - mean {dwi_pre}/{b0_src}.mif -axis 3 -force -quiet 
!mrconvert {dwi_pre}/{b0_src}.mif {dwi_pre}/{b0_src}.nii.gz -force -quiet

In [ ]:
import cmd_median_otsu as cmo
mo_src = dwi_pre + '/' + b0_src + '.nii.gz'
mo_out = dwi_pre + '/' + b0_otsu + '.nii.gz'
mo_mask = dwi_pre + '/' + dwi_nodif + '.nii.gz'
cmo.cmd_median_otsu(mo_src, mo_out, mo_mask)
!mrconvert {dwi_pre}/{b0_otsu}.nii.gz {dwi_pre}/{b0_otsu}.mif -force -quiet
!mrconvert {dwi_pre}/{dwi_nodif}.nii.gz {dwi_pre}/{dwi_nodif}.mif -force -quiet

Eddy correction

In [ ]:
!eddy_correct {dwi_dir}/{dwi_src}.nii.gz {dwi_pre}/{dwi_ecc}.nii.gz 0

In [26]:
!mrconvert {dwi_pre}/{dwi_ecc}.nii.gz {dwi_pre}/{dwi_ecc}.mif \
    -fslgrad {dwi_dir}/{dwi_src}.bvecs {dwi_dir}/{dwi_src}.bvals \
    -datatype float32 -stride 0,0,0,1 -nthreads 5 -force -quiet

In [36]:
!dwiextract  {dwi_pre}/{dwi_ecc}.mif - -shell 0 -quiet | \
    mrmath - mean {dwi_pre}/{b0_ecc}.mif -axis 3 -force -quiet
!mrconvert {dwi_pre}/{b0_ecc}.mif {dwi_pre}/{b0_ecc}.nii.gz -force -quiet

#### Import structural data

In [ ]:
!cp {dicom_t1w}/{t1w_src} {t1w_dir}/{t1w_src}

#### Brain extraction with 'bet'

In [ ]:
!bet {t1w_dir}/{t1w_src} {t1w_pre}/{t1w_bet} -R -m

#### Brain structure segmentation with FSL

In [31]:
!5ttgen fsl {t1w_pre}/{t1w_bet} {t1w_pre}/{t1w_5tt}.mif -premasked -nocrop -force -quiet

In [33]:
!mrconvert {t1w_pre}/{t1w_5tt}.mif {t1w_pre}/{t1w_5tt}.nii.gz -force -quiet

#### Brain structure segmentation with FREESURFER

In [ ]:
!SUBJECTS_DIR={fs_dir}; recon-all -all -i {t1w_dir}/{t1w_src} -s freesurfer -parallel

#### White Matter mask segmentation with FSL

In [34]:
tmpdir = '_tmp_wm'
!mkdir -p {tmpdir}
!fslsplit {t1w_pre}/{t1w_5tt}.nii.gz {tmpdir}/wm_ -t 
!mv {tmpdir}/wm_0002.nii.gz {t1w_pre}/{t1w_fsl_wmm}
!rm -r {tmpdir}

In [47]:
!fslmaths {dwi_pre}/{b0_ecc}.nii.gz \
    -mas {dwi_pre}/{dwi_nodif}\
    {dwi_pre}/{b0_ecc}_bet.nii.gz
!flirt -in {t1w_pre}/{t1w_fsl_wmm} \
    -ref {dwi_pre}/{b0_ecc}_bet.nii.gz \
    -omat {t1w_pre}/{b0_fsl_omat} 
!flirt -in {t1w_pre}/{t1w_fsl_wmm} \
    -ref {dwi_pre}/{b0_ecc}_bet.nii.gz \
    -applyxfm -init {t1w_pre}/{b0_fsl_omat} \
    -out {t1w_pre}/{dwi_fsl_wmm} \
    -interp nearestneighbour

#### White Matter mask segmentation with FREESURFER
(Warning: implementation not completed)

In [ ]:
from wm_from_aparc_aseg import wm_from_aparc_aseg
!mri_convert {fs_aparc_aseg} {t1w_pre}/{aparc_aseg}
wdir = 'derivatives/preprocess/sub-%s/anat' % sub
t1w_wmm = t1w_pre + '/' + 'sub-%s_var-fs_space-T1w_class-wm_dtissue.nii.gz' % sub
aparc_aseg = fs_dir + '/' + 
wm_from_aparc_aseg(aparc_aseg, t1w_wmm )

In [ ]:
omat = 'sub-%s_var-fs_affine.mat' % sub
dwi_wmm = 'sub-%s_var-fs_space-dwi_class-wm_dtissue.nii.gz' % sub
b0 = 'sub-%s/dwi/sub-%s_var-b0_dwi.nii.gz' % (sub, sub)
!flirt -in {wdir}/{t1w_wmm} \
    -ref {b0} \
    -omat {wdir}/{omat} 
!flirt -in {wdir}/{t1w_wmm} \
    -ref {b0} \
    -applyxfm -init {wdir}/{omat} \
    -out {wdir}/{dwi_wmm} \
    -interp nearestneighbour

#### fMRI ROI Registration
(it looks like T1 and DWI are already aligned)

In [48]:
#!flirt -in {roi_dir}/{roi_all}.nii.gz \
#    -ref {dwi_pre}/{b0_ecc}_bet.nii.gz \
#    -applyxfm -init {t1w_pre}/{b0_fsl_omat} \
#    -out {trk_dir}/{roi_all_dwi}.nii.gz \
#    -interp nearestneighbour

#### Model Reconstruction

In [48]:
# Choice of white matter mask
wmm_ref = dwi_fsl_wmm

In [ ]:
# Computation of mask for response estimate
!dwi2tensor {dwi_pre}/{dwi_ecc}.mif \
    {trk_dir}/{dti}.mif 
!tensor2metric -fa {trk_dir}/{fa}.mif \
    {trk_dir}/{dti}.mif \
    -mask {dwi_pre}/{wmm_ref}

In [ ]:
# Estimate of response function
!maskfilter {dwi_pre}/{wmm_ref} erode -npass 3 - | \
    mrmath {dwi_pre}/{wmm_ref} - product - | \
    mrthreshold - -abs 0.7 {trk_dir}/{sf}.mif -force
!dwi2response tournier {dwi_pre}/{dwi_ecc}.mif \
    -mask {trk_dir}/{sf}.mif \
    {trk_dir}/{rf} \
    -force -quiet

In [55]:
# Computation of the constraint spherical deconvolution
!dwi2fod csd {dwi_pre}/{dwi_ecc}.mif \
    {trk_dir}/{rf} \
    -mask {dwi_pre}/{wmm_ref} \
    {trk_dir}/{odf}.mif \
    -force -quiet

dwi2fod: [100%] uncompressing image "derivatives/preprocess/sub-00/sub-00_var-fsl_space-dwi_class-wm_dtissue.nii.gz"
dwi2fod: [100%] performing constrained spherical deconvolution


#### Tracking

In [54]:
# Tuning of ROI for seeding tractography
!mrconvert {roi_dir}/{roi_all}.nii.gz {trk_dir}/{roi_all}.mif \
    -force -quiet
!maskfilter -npass 2 \
    {trk_dir}/{roi_all}.mif \
    dilate \
    {trk_dir}/{roi_all_dwi_exp}.mif \
    -force -quiet
!mrconvert {trk_dir}/{roi_all_dwi_exp}.mif \
    {trk_dir}/{roi_all_dwi_exp}.nii.gz \
    -force -quiet

In [66]:
# Computation of probabilistic tracking
tck_step = 0.5
tck_cutoff = 0.1
tck_number = 1000
tck_maxlength = 250
odf_src = trk_dir + '/' + odf + '.mif'
tck_out = trk_dir + '/' + tck_prob + '.tck'
roi_seed = trk_dir + '/' + roi_all_dwi_exp + '.mif'
!tckgen {odf_src} {tck_out} \
    -algorithm iFOD2 \
    -maxlength 250 \
    -cutoff 0.1 \
    -number 1000000 \
    -seed_image {roi_seed} \
    -force

tckgen: [WARNING] existing output files will be overwritten
tckgen: [100%]  1312733 generated,  1000000 selected


#### Connectome

In [63]:
# Build the node map of ROIs
tmpdir = '_tmp_sgm'
!mkdir -p {tmpdir}
!fslmaths {roi_dir}/{roi_1} -mul 1 {tmpdir}/roi_sgm.nii.gz
!fslmaths {roi_dir}/{roi_2} -mul 2 -add {tmpdir}/roi_sgm.nii.gz {tmpdir}/roi_sgm.nii.gz
!fslmaths {roi_dir}/{roi_3} -mul 3 -add {tmpdir}/roi_sgm.nii.gz {tmpdir}/roi_sgm.nii.gz
!fslmaths {roi_dir}/{roi_4} -mul 4 -add {tmpdir}/roi_sgm.nii.gz {tmpdir}/roi_sgm.nii.gz
!mrconvert {tmpdir}/roi_sgm.nii.gz {roi_dir}/{roi_sgm} -force -quiet
!rm -r {tmpdir}

In [67]:
# Compute the connectome as adjacency matrix
tck_out = trk_dir + '/' + tck_prob + '.tck'
sgm_src = roi_dir + '/' + roi_sgm
con_out = con_dir + '/' + con_csv
!tck2connectome {tck_out} {sgm_src} {con_out} -zero_diagonal -force

tck2connectome: [WARNING] existing output files will be overwritten
tck2connectome: [100%] Constructing connectome


In [68]:
!cat {con_dir}/{con_csv}

0 53 79 15
0 0 1 1705
0 0 0 12
0 0 0 0
